In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import urllib.request
import datetime
import json
import glob
import sys
import os

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

pd.options.display.float_format = '{:.2f}'.format

In [2]:
from selenium import webdriver
import chromedriver_autoinstaller
    
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')

driver.implicitly_wait(10)
driver.get('http://surffing.net/MainSeoSearch.do')

In [3]:
search=driver.find_element(By.ID,'saerchKeyword')
search.send_keys('제로웨이스트')
search.send_keys(Keys.ENTER)

In [4]:
title=driver.find_elements(By.CSS_SELECTOR,'tr>td.center:nth-child(1)')

In [5]:
title_l=[]

for i in range(len(title)):
    title_l.append(title[i].text)
    print(title_l[i])

제로웨이스트
친환경신발
제주도어린이체험
업사이클링제품
친환경제품
업사이클링
친환경운동화
제주도수학여행
업사이클링브랜드
친환경의류
지구의날행사
르무통
업사이클링가방
친환경브랜드
리사이클링
페트병재활용


In [6]:
class NaverDataLabOpenAPI():
    """
    네이버 데이터랩 오픈 API 컨트롤러 클래스
    """

    def __init__(self, client_id, client_secret):
        """
        인증키 설정 및 검색어 그룹 초기화
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.keywordGroups = []
        self.url = "https://openapi.naver.com/v1/datalab/search"

    def add_keyword_groups(self, group_dict):
        """
        검색어 그룹 추가
        """

        keyword_gorup = {
            'groupName': group_dict['groupName'],
            'keywords': group_dict['keywords']
        }
        
        self.keywordGroups.append(keyword_gorup)
        print(f">>> Num of keywordGroups: {len(self.keywordGroups)}")
        
    def get_data(self, startDate, endDate, timeUnit, device, ages, gender):
        """
        요청 결과 반환
        timeUnit - 'date', 'week', 'month'
        device - None, 'pc', 'mo'
        ages = [], ['1' ~ '11']
        gender = None, 'm', 'f'
        """

        # Request body
        body = json.dumps({
            "startDate": startDate,
            "endDate": endDate,
            "timeUnit": timeUnit,
            "keywordGroups": self.keywordGroups,
            "device": device,
            "ages": ages,
            "gender": gender
        }, ensure_ascii=False)
        
        # Results
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id",self.client_id)
        request.add_header("X-Naver-Client-Secret",self.client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            # Json Result
            result = json.loads(response.read())
            
            period = []
            ratio = []
            title = []
            
            for val in result['results']:
                period.append(val['data'][0]['period'])
                ratio.append(val['data'][0]['ratio'])
                title.append(val['title'])
            
            df = pd.DataFrame({"title" : title, 'period' : period, "ratio" : ratio})
            #             pprint(result)
#             df = pd.DataFrame({'A': ['a','a','a','a','a']})
#             for i in range(len(self.keywordGroups)):
#                 tt = pd.DataFrame(result['results'][i]['keywords'])
#                 tmp = pd.DataFrame(result['results'][i]['data'])
#                 df = pd.concat([tmp,df,tt])
#                 print(tt)
#                 print(tmp)
                
            
        else:
            print("Error Code:" + rescode)
            
        return df

In [7]:
df

NameError: name 'df' is not defined

In [9]:
keyword_group_set = {
    'keyword_group_1': {'groupName': "애플", 'keywords': ["애플"]},
    'keyword_group_2': {'groupName': "아마존", 'keywords': ["아마존"]},
    'keyword_group_3': {'groupName': "구글", 'keywords': ["구글"]},
    'keyword_group_4': {'groupName': "테슬라", 'keywords': ["테슬라"]},
    'keyword_group_5': {'groupName': "페이스북", 'keywords': ["페이스북"]}
}

# k_title = pd.DataFrame(keyword_group_set['keyword_group_1']['keywords'])
# # print(keyword_group_set)
# print(k_title)

In [10]:
for idx, (key, val) in enumerate(keyword_group_set.items()):
    val['groupName'] = title_l[idx]
    val['keywords'] = title_l[idx]
    print(idx, key, val)

0 keyword_group_1 {'groupName': '제로웨이스트', 'keywords': '제로웨이스트'}
1 keyword_group_2 {'groupName': '친환경신발', 'keywords': '친환경신발'}
2 keyword_group_3 {'groupName': '제주도어린이체험', 'keywords': '제주도어린이체험'}
3 keyword_group_4 {'groupName': '업사이클링제품', 'keywords': '업사이클링제품'}
4 keyword_group_5 {'groupName': '친환경제품', 'keywords': '친환경제품'}


In [10]:
for idx, (key, val) in enumerate(keyword_group_set.items()):
#     val['groupName'] = title_l[]
    print(idx, key, val)

0 keyword_group_1 {'groupName': '제로웨이스트', 'keywords': '제로웨이스트'}
1 keyword_group_2 {'groupName': '친환경신발', 'keywords': '친환경신발'}
2 keyword_group_3 {'groupName': '제주도어린이체험', 'keywords': '제주도어린이체험'}
3 keyword_group_4 {'groupName': '업사이클링제품', 'keywords': '업사이클링제품'}
4 keyword_group_5 {'groupName': '친환경제품', 'keywords': '친환경제품'}


In [11]:
# API 인증 정보 설정
client_id = "xTsOEpqU869lYEaqyaU6"
client_secret = "kkp_ZP2j9T"

# 요청 파라미터 설정
startDate = "2020-01-01"
endDate = "2020-01-31"
timeUnit = 'month'
device = ''
ages = []
gender = ''

# 데이터 프레임 정의
naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)

naver.add_keyword_groups(keyword_group_set['keyword_group_1'])
naver.add_keyword_groups(keyword_group_set['keyword_group_2'])
naver.add_keyword_groups(keyword_group_set['keyword_group_3'])
naver.add_keyword_groups(keyword_group_set['keyword_group_4'])
naver.add_keyword_groups(keyword_group_set['keyword_group_5'])

df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)
df

>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 2
>>> Num of keywordGroups: 3
>>> Num of keywordGroups: 4
>>> Num of keywordGroups: 5


HTTPError: HTTP Error 400: Bad Request